### **1.Setup**


In [12]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import re
import numpy as np 
import matplotlib.pyplot as plt 
import pathlib

import tensorflow as tf 
import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization

keras.utils.set_random_seed(111)

### **2.Download the dataset**

In [13]:
# text_file = keras.utils.get_file(
#     fname="Flickr8k_Dataset.zip",
#     origin="https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip",
#     extract=True # ファイルを展開する
# )
# image_file = keras.utils.get_file(
#     fname="Flickr8k_text.zip",
#     origin="https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip",
#     extract=True # ファイルを展開する
# )
# text_file = pathlib.Path(text_file).parent / "Flickr8k_text" / "Flickr8k_text.txt"

In [14]:
# Path to the images
IMAGES_PATH = "./data/Flickr8k_Dataset"

# 画像の寸法
IMAGE_SIZE = (299,299)

# 語彙のサイズ
VOCAB_SIZE = 10000

# 固定長
SEQ_LENGTH = 25

# 画像埋め込みとトークン埋め込みの次元
ENBED_DIM = 512

# Feed-forwardネットワークのレイヤー単位
FF_DIM = 64

# 他のトレーニングパラメータ
BATCH_SIZE = 64
EPOCHS = 30
AUTOTUNE = tf.data.AUTOTUNE

### **3.Preparing the dataset**

In [ ]:
def load_captions_data(filename):
    """キャプション（テキスト）データを読み込み、対応する画像にマッピングする
    
    Args:
        filename: キャプションデータを含むテキストファイルへのパス
        
    Returns:
        caption_mapping: 画像名と対応するキャプションをマッピングするディクショナリ
        text_data: 利用可能なすべてのキャプションを含むリスト
    """

    with open(filename) as caption_file:
        caption_data = caption_file.readlines()
        caption_mapping = {}
        text_data = []
        images_to_skip = set()

        for line in caption_data:
            line = line.rstrip("\n")
            # 画像名とキャプションはタブで区切られる
            img_name, caption = line.split("\t")

            # 各画像は、5つの異なるキャプションのために5回繰り返される。
            # 各画像には接頭辞`#(caption_number)`が付く。
            img_name = img_name.split("#")[0]
            img_name = os.path.join(IMAGES_PATH, img_name.strip())

            # 短すぎるキャプションや長すぎるキャプションは削除する
            tokens = caption.strip().split()

            if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
                images_to_skip.add(img_name)
                continue

            if img_name.endswith("jpg") and img_name not in images_to_skip:
                # それぞれのキャプチョンに開始トークンと終了トークンを追加する
                caption = "<start> " + caption.strip() + " <end>"
                text_data.append(caption)

                if img_name in caption_mapping:
                    caption_mapping[img_name].append(caption)
                else:
                    caption_mapping[img_name] = [caption]
        
        for img_name in images_to_skip:
            if img_name in caption_mapping:
                del caption_mapping[img_name]

        return caption_mapping, text_data
    

def train_val_split(caption_data, train_size = 0.8, shuffle=True):
    """キャプションデータセットを訓練と検証データセットに分ける
    
    Args:
        caption_data (dict): マッピングされたキャプションデータを含むディクショナリ
        train_size (float): 全データセットのうち、トレーニングデータとして使用するデータセットの割合
        shufflw (bool): 分割前にデータセットをシャッフルするかどうか
        
    Returns:
        訓練データセットと検証データセットを、2つの分離されたディクショナリとして扱う
    """